In [1]:
from QAOAUtils import * 
from LieUtils import *

In [2]:
def qaoa_cost(params, precomp, mixer_ops, init_state):
    psi = QAOA_eval(precomp, params, mixer_ops, init_state)
    return np.real(np.dot(np.conj(psi), precomp * psi))

In [3]:
from scipy.optimize import minimize

def BFGS_optimize(precomp, mixer_ops, init_state, p=1):
    init_params = np.random.uniform(0, np.pi, size= 2 * p)
    cost_fn = lambda x: qaoa_cost(x, precomp, mixer_ops, init_state)

    result = minimize(cost_fn, init_params, method='BFGS', options={'maxiter': 10})
    return result.x 

In [4]:
from scipy.stats import gmean

def estimate_basin_radius(min_params, cost_fn, precision=1e-2, epsilon=1e-2):
    n = len(min_params)
    directions = np.linalg.qr(np.random.randn(n, n))[0].T  # orthonormal directions
    radii = []

    for d in directions:
        l, r = 0, np.pi
        while r - l > precision:
            mid = (l + r) / 2
            perturbed = min_params + d * mid
            result = minimize(cost_fn, perturbed, method='BFGS', options={'maxiter': 10})
            if np.linalg.norm(result.x - min_params) < epsilon:
                l = mid
            else:
                r = mid
        radii.append(l)
    
    return gmean(radii)

In [5]:
def estimate_num_minima(all_radii, param_dim):
    mean_radius = gmean(all_radii)
    
    V_total = (2 * np.pi)**param_dim
    basin_vol = (np.pi ** (param_dim / 2)) / np.math.gamma(param_dim / 2 + 1) * mean_radius ** param_dim
    
    return V_total / basin_vol

In [8]:
def local_minima(precomp, GW2_circ_data, GW3_circ_data, A, GW2_angles, GW3_angles):
    pickle_file = 'LocalMinimaRuns.pkl'
    
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as f:
            all_runs = pickle.load(f)
    else:
        all_runs = []

    ### BEGIN: Estimate basin radii and number of minima
    all_radii = []
    print(1)
    for init_angles, circ_data in [(GW2_angles, GW2_circ_data), (GW3_angles, GW3_circ_data)]:
        print(2)
        min_params = BFGS_optimize(precomp, circ_data[1], circ_data[0], p=len(circ_data[1](0.5)))
        print(3)
        cost_fn = lambda x: qaoa_cost(x, precomp, circ_data[1], circ_data[0])
        print(4)
        radius = estimate_basin_radius(min_params, cost_fn)
        print(5)
        all_radii.append(radius)
        print(6)
    
    num_minima_est = estimate_num_minima(all_radii, len(min_params))
    ### END

    all_runs.append({
        'A': A,
        'GW2_angles': GW2_angles,
        'GW3_angles': GW3_angles,
        'basin_radii': all_radii,
        'estimated_minima': num_minima_est
    })
    
    with open(pickle_file, 'wb') as f:
        pickle.dump(all_runs, f)


In [13]:
from tqdm import tqdm

for _ in tqdm(range(20), "Overall Progress"):
    n = 4
    A = rand_adj(n)
    
    Y = GW(A)  ### full GW embedding
    _, GW2_angles, _ = GW2(A, GW_Y=Y)
    _, GW3_angles, _ = GW3(A, GW_Y=Y)
    
    GW2_circ_data = Q2_data(GW2_angles, rotation=0)
    GW3_circ_data = Q3_data(GW3_angles, rotation=0)

    precomp = pre_compute(A)  ### Hamiltonian info

    local_minima(precomp, GW2_circ_data, GW3_circ_data, A, GW2_angles, GW3_angles)


Overall Progress:   0%|                                                                         | 0/20 [00:00<?, ?it/s]

1
2
3
4
5
6
2
3
4


C:\Users\Micha\AppData\Local\Temp\ipykernel_7836\1489087418.py:5: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  basin_vol = (np.pi ** (param_dim / 2)) / np.math.gamma(param_dim / 2 + 1) * mean_radius ** param_dim
Overall Progress:   5%|███▎                                                             | 1/20 [00:03<01:00,  3.19s/it]

5
6
Estimated number of local minima: 2.9785946427344433e+23
1
2
3
4
5
6
2
3
4


C:\Users\Micha\AppData\Local\Temp\ipykernel_7836\1489087418.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  return V_total / basin_vol
Overall Progress:  10%|██████▌                                                          | 2/20 [00:55<09:32, 31.80s/it]

...

5
6
Estimated number of local minima: inf
1
2
3
4
5
6
2
3
4


Overall Progress:  95%|████████████████████████████████████████████████████████████▊   | 19/20 [06:10<00:13, 13.28s/it]

5
6
Estimated number of local minima: 2.9785946427344433e+23
1
2
3
4
5
6
2
3
4


Overall Progress: 100%|████████████████████████████████████████████████████████████████| 20/20 [06:53<00:00, 20.69s/it]

5
6
Estimated number of local minima: inf


In [17]:
pickle_file = 'LocalMinimaRuns.pkl'
with open(pickle_file, 'rb') as f:
    all_runs = pickle.load(f)

for a in all_runs:
    print(a["estimated_minima"])

2.9785946427344433e+23
2.9785946427344433e+23
inf
inf
inf
inf
inf
2.9785946427344433e+23
2.9785946427344433e+23
2.9785946427344433e+23
inf
2.9785946427344433e+23
2.9785946427344433e+23
2.9785946427344433e+23
inf
2.9785946427344433e+23
2.9785946427344433e+23
2.9785946427344433e+23
inf
2.9785946427344433e+23
inf


In [ ]:
radii_over_runs = [gmean(run['basin_radii']) for run in all_runs]
minima_over_runs = [run['estimated_minima'] for run in all_runs]

plt.plot(radii_over_runs, label='Mean Basin Radius')
plt.xlabel('Run #')
plt.ylabel('Radius')
plt.title('Basin Radius Over Runs')
plt.legend()
plt.show()

plt.plot(minima_over_runs, label='Estimated Minima')
plt.yscale('log')
plt.xlabel('Run #')
plt.ylabel('Num Minima')
plt.title('Estimated Number of Minima Over Runs')
plt.legend()
plt.show()
